# Comparing CLMM to NumCosmo
This notebook compares `CLMM` to `NumCosmo`.

In [ ]:
import numpy as np
import cluster_toolkit as ct
from astropy import constants, cosmology, units
import math

In [ ]:
# For NumCosmo
import gi

gi.require_version("NumCosmo", "1.0")
gi.require_version("NumCosmoMath", "1.0")
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

In [ ]:
import sys
import os

os.environ["CLMM_MODELING_BACKEND"] = "ccl"

from clmm import theory as pp

## Configuration

### Cosmology

In [ ]:
# If CCL is installed, define a CCL cosmology object
# import pyccl as ccl
# cosmo_ccl = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96)

# If CCL is not installed, use astropy instead with the following lines
# from astropy.cosmology import FlatLambdaCDM
# astropy_cosmology_object = FlatLambdaCDM(H0=67.66, Odm0=0.262, Ob0=0.049)
# cosmo_ccl = pp.cclify_astropy_cosmo(astropy_cosmology_object)

cosmo_ccl = pp.Cosmology(H0=67.66, Omega_dm0=0.262, Omega_b0=0.049)

cosmo_ccl.be_cosmo

print(cosmo_ccl["Omega_m0"], 0.262 + 0.049)

### Cluster/source config

In [ ]:
# Select density profile and profile parametrization options
density_profile_parametrization = "nfw"
mass_Delta = 200  # Mass defintion: mean matter density
cluster_mass = 1.0e15  # Msun
cluster_concentration = 4.0
z_max = 3.0
z_cluster = 1.0
z_src = 2.0

### Vectors for inputs

In [ ]:
z_a = np.linspace(0.01, 10.0, 1000)
a_a = 1.0 / (1.0 + z_a)
r3d = np.logspace(-2, 2, 100)

### NumCosmo
Pass configs to `NumCosmo`

In [ ]:
#########################
# Initializing NumCosmo #
#########################
Ncm.cfg_init()

##################
# Set Cosmology ##
##################
cosmo = Nc.HICosmoDEXcdm(massnu_length=1)
cosmo.omega_x2omega_k()

cosmo.props.w = -1.0
cosmo.props.H0 = cosmo_ccl["H0"]

cosmo.param_set_by_name("H0", cosmo_ccl["H0"])
cosmo.param_set_by_name("Omegab", cosmo_ccl["Omega_b0"])
cosmo.param_set_by_name("Omegac", cosmo_ccl["Omega_dm0"])
cosmo.param_set_by_name("w", -1.0)
cosmo.param_set_by_name("Omegak", 0.0)
cosmo.param_set_by_name("Tgamma0", 2.7255)
cosmo.param_set_by_name("massnu_0", 0.06)
# cosmo_nc.param_set_by_name ("Tnu_0",    (4.0/11.0)**(1.0/3.0))
# cosmo_nc.param_set_by_name ("gnu_0",    1.0153080198436308)
cosmo.param_set_by_name(
    "ENnu",
    3.046
    - 3.0
    * cosmo.E2Press_mnu(1.0e10)
    / (cosmo.E2Omega_g(1.0e10) * (7.0 / 8.0 * (4.0 / 11.0) ** (4.0 / 3.0))),
)

##########################
# Config Cluster/Source ##
##########################

# NumCosmo Density profile
hms = Nc.HaloCMParam.new(Nc.HaloMassSummaryMassDef.MEAN, mass_Delta)
dp = Nc.HaloDensityProfileNFW.new(hms)
hms.props.log10MDelta = math.log10(cluster_mass)
hms.props.cDelta = cluster_concentration

dist = Nc.Distance.new(z_max)
smd = Nc.WLSurfaceMassDensity.new(dist)

smd.prepare(cosmo)

print(cosmo.Omega_m0() / cosmo_ccl["Omega_m0"] - 1.0)

import pyccl as ccl

z = 10.0
A = ccl.background.omega_x(cosmo_ccl.be_cosmo, 1.0 / (1.0 + z), "matter")
B = cosmo.E2Omega_m(z) / cosmo.E2(z)

print(A / B - 1.0)

## Comparisons

### Function to make automatic comparisons

In [ ]:
import pylab as plt


def compare(x, y_clmm, y_nc, x_name="x", y_name="func", subplots_pars={"figsize": (6, 4)}):
    clmm_name, nc_name = "%s_{clmm}" % y_name, "%s_{nc}" % y_name
    diff = y_nc / y_clmm - 1.0
    print("rel diff min: %g\nrel diff max: %g" % (min(abs(diff)), max(abs(diff))))
    fig, axs = plt.subplots(2, sharex=True, **subplots_pars)
    fig.subplots_adjust(hspace=0)

    axs[0].plot(x, y_clmm, label="clmm", lw=3)
    axs[0].plot(x, y_nc, label="nc")
    axs[1].plot(x, diff, c="r")
    axs[1].axhline(0, ls=":", c="0", lw=0.5)

    axs[0].legend()
    axs[0].set_ylabel("$%s$" % y_name)
    axs[1].set_xlabel("$%s$" % x_name)
    axs[1].set_ylabel("$%s/%s-1$" % (nc_name, clmm_name))

    return fig, axs

### Quick test of all theory function

In [ ]:
Da_a = cosmo_ccl.eval_da_a1a2(a_a)
nc_Da_a = np.array([dist.angular_diameter(cosmo, z) * cosmo.RH_Mpc() for z in z_a])

fig, axs = compare(z_a, Da_a, nc_Da_a, x_name="redshift", y_name="D_a(z)")
axs[0].set_yscale("log")

In [ ]:
rho = pp.compute_3d_density(
    r3d,
    mdelta=cluster_mass,
    cdelta=cluster_concentration,
    z_cl=z_cluster,
    cosmo=cosmo_ccl,
    delta_mdef=mass_Delta,
)
nc_rho = np.array([dp.eval_density(cosmo, r3d_i, z_cluster) for r3d_i in r3d])

fig, axs = compare(r3d, rho, nc_rho, x_name="radius [Mpc]", y_name="\\rho")
axs[0].set_xscale("log")

In [ ]:
Sigma = pp.compute_surface_density(
    r3d,
    cluster_mass,
    cluster_concentration,
    z_cl=z_cluster,
    cosmo=cosmo_ccl,
    delta_mdef=mass_Delta,
    halo_profile_model="nfw",
)
nc_Sigma = np.array([smd.sigma(dp, cosmo, r3d_i, z_cluster) for r3d_i in r3d])

fig, axs = compare(r3d, Sigma, nc_Sigma, x_name="radius [Mpc]", y_name="\\Sigma")
axs[0].set_xscale("log")

In [ ]:
DeltaSigma = pp.compute_excess_surface_density(
    r3d,
    cluster_mass,
    cluster_concentration,
    z_cl=z_cluster,
    cosmo=cosmo_ccl,
    delta_mdef=mass_Delta,
    halo_profile_model="nfw",
)

nc_SigmaMean = np.array([smd.sigma_mean(dp, cosmo, r3d_i, z_cluster) for r3d_i in r3d])
nc_DeltaSigma = nc_SigmaMean - nc_Sigma

fig, axs = compare(r3d, DeltaSigma, nc_DeltaSigma, x_name="radius [Mpc]", y_name=r"\Delta\Sigma")
axs[0].set_xscale("log")

In [ ]:
Sigmac = cosmo_ccl.eval_sigma_crit(z_len=z_cluster, z_src=z_src)
nc_Sigmac = smd.sigma_critical(cosmo, z_src, z_cluster, z_cluster)

print("% 22.15g % 22.15g %e" % (Sigmac, nc_Sigmac, Sigmac / nc_Sigmac - 1.0))

In [ ]:
gammat = pp.compute_tangential_shear(
    r3d,
    mdelta=cluster_mass,
    cdelta=cluster_concentration,
    z_cluster=z_cluster,
    z_src=z_src,
    cosmo=cosmo_ccl,
    delta_mdef=mass_Delta,
    halo_profile_model="nfw",
    z_src_info="discrete",
)

nc_gammat = np.array([smd.shear(dp, cosmo, r3d_i, z_src, z_cluster, z_cluster) for r3d_i in r3d])

fig, axs = compare(r3d, gammat, nc_gammat, x_name="radius [mpc]", y_name=r"{\gamma_t}")
axs[0].set_xscale("log")

In [ ]:
kappa = pp.compute_convergence(
    r3d,
    mdelta=cluster_mass,
    cdelta=cluster_concentration,
    z_cluster=z_cluster,
    z_src=z_src,
    cosmo=cosmo_ccl,
    delta_mdef=mass_Delta,
    halo_profile_model="nfw",
    z_src_info="discrete",
)

nc_kappa = np.array(
    [smd.convergence(dp, cosmo, r3d_i, z_src, z_cluster, z_cluster) for r3d_i in r3d]
)

fig, axs = compare(r3d, kappa, nc_kappa, x_name="radius [mpc]", y_name=r"\kappa")
axs[0].set_xscale("log")

In [ ]:
gt = pp.compute_reduced_tangential_shear(
    r3d,
    mdelta=cluster_mass,
    cdelta=cluster_concentration,
    z_cluster=z_cluster,
    z_src=z_src,
    cosmo=cosmo_ccl,
    delta_mdef=mass_Delta,
    halo_profile_model="nfw",
    z_src_info="discrete",
)
nc_gt = np.array(
    [smd.reduced_shear(dp, cosmo, r3d_i, z_src, z_cluster, z_cluster) for r3d_i in r3d]
)

fig, axs = compare(r3d, gt, nc_gt, x_name="radius [mpc]", y_name=r"{g_t}")
axs[0].set_xscale("log")
axs[0].set_yscale("symlog")

In [ ]:
nc_mu = np.array(
    [smd.magnification(dp, cosmo, r3d_i, z_src, z_cluster, z_cluster) for r3d_i in r3d]
)

### Save the numcosmo outputs

In [ ]:
np.savetxt(
    "radial_profiles.txt",
    np.array([r3d, nc_rho, nc_Sigma, nc_DeltaSigma, nc_gammat, nc_kappa, nc_gt, nc_mu]).T,
    header="r3d  rho Sigma DeltaSigma gammat kappa gt mu",
)

In [ ]:
np.savetxt("angular_diameter_distance.txt", np.array([a_a, z_a, nc_Da_a]).T, header="a_a z_a Da_a")

In [ ]:
config_dict = {
    "dl": dist.angular_diameter(cosmo, z_cluster) * cosmo.RH_Mpc(),
    "z_src": z_src,
    "cluster_mass": cluster_mass,
    "G[m3/km.s2]": Ncm.C.G(),
    "aexp_cluster": 1.0 / (1.0 + z_cluster),
    "density_profile_parametrization": density_profile_parametrization,
    "ds": dist.angular_diameter(cosmo, z_src) * cosmo.RH_Mpc(),
    "cosmo_Ob0": cosmo_ccl["Omega_b0"],
    "aexp_source": 1.0 / (1.0 + z_src),
    "dsl": dist.angular_diameter_z1_z2(cosmo, z_cluster, z_src) * cosmo.RH_Mpc(),
    "z_cluster": z_cluster,
    "mass_Delta": mass_Delta,
    "lightspeed[km/s]": Ncm.C.c() / 1000.0,
    "cosmo_H0": cosmo_ccl["H0"],
    "cluster_concentration": cluster_concentration,
    "cosmo_Om0": cosmo_ccl["Omega_m0"],
    "cosmo_Odm0": cosmo_ccl["Omega_dm0"],
    "nc_Sigmac": nc_Sigmac,
    "Msun[kg]": Ncm.C.mass_solar(),
    "pc_to_m": Ncm.C.pc(),
}

import json

with open("config.json", "w") as fp:
    json.dump(config_dict, fp)